# Environment setup

In [ ]:
!sudo apt update
# vector database
%pip install faiss-gpu
# PDF
!apt install poppler-utils
!apt install tesseract-ocr
!apt install libtesseract-dev
%pip install langchain-community
%pip install langchain-unstructured
%pip install langchain-huggingface
%pip install nltk --upgrade
%pip install unstructured[pdf]
%pip install optimum
%pip install auto-gptq

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [60.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,632 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packa

In [ ]:
# Clone our repo to obtain the data
!git clone https://github.com/andrzejczukm/NLP-RAG.git

fatal: destination path 'NLP-RAG' already exists and is not an empty directory.


# Loading data

In [ ]:
import os
import itertools

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from langchain_core.documents import Document
# from langchain_unstructured import UnstructuredLoader
from langchain.document_loaders.unstructured import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from unstructured.cleaners.core import clean_extra_whitespace, group_broken_paragraphs
from sentence_transformers import CrossEncoder
from transformers import pipeline, set_seed as transformers_set_seed

from google.colab import userdata

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# set seeds for reproducibility
seed = 3
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
transformers_set_seed(seed)

In [ ]:
INPUT_PATH = './NLP-RAG/data'

In [ ]:
post_processors = [clean_extra_whitespace, group_broken_paragraphs]

In [ ]:
documents_loaders: list[UnstructuredFileLoader] = []

for filename in os.listdir(INPUT_PATH):
    if not filename.endswith('ulysses.pdf'):
        continue

    documents_loaders.append(UnstructuredFileLoader(
        os.path.join(INPUT_PATH, filename),
        post_processors=post_processors,
        paragraph_grouper=group_broken_paragraphs,
    ))

<ipython-input-6-91dfcc8c92b7>:7: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-unstructured package and should be used instead. To use it run `pip install -U :class:`~langchain-unstructured` and import as `from :class:`~langchain_unstructured import UnstructuredLoader``.
  documents_loaders.append(UnstructuredFileLoader(


In [ ]:
chunk_size = 2000

text_splitter = RecursiveCharacterTextSplitter(
    separators=['\n\n', '.', '?', '\n'],
    chunk_size=chunk_size,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
    keep_separator="end"
)

In [ ]:
# ~ 1 min (but note there is just one document for PoC)
passages: dict[str, list[Document]] = {}

for document in documents_loaders:
    passages_for_doc = document.load_and_split(text_splitter)
    passages[document.file_path] = passages_for_doc

In [ ]:
# merging all documents into a single list
passages_list = list(itertools.chain(*passages.values()))

In [ ]:
# example passage from the middle
passages_list[len(passages_list) // 2]

Document(metadata={'source': './NLP-RAG/data/joyce_ulysses.pdf'}, page_content='Glad to get away from other chap’s wife. Eating off his cold plate. Chap in the Burton today spitting back gumchewed gristle. French letter still in my pocketbook. Cause of half the trouble. But might happen sometime, I don’t think. Come in, all is prepared. I dreamt. What? Worst is beginning. How they change the venue when it’s not what they like. Ask you do you like mushrooms because she once knew a gentleman who. Or ask you what someone was going to say when he changed his mind and stopped. Yet if I went the whole hog, say: I want to, something like that. Because I did. She too. Offend her. Then make it up. Pretend to want something awfully, then cry off for her sake. Flatters them. She must have been thinking of someone else all the time. What harm? Must since she came to the use of reason, he, he and he. First kiss does the trick. The propitious moment. Something inside them goes pop. Mushy like, tell 

# Proof of concept

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Embedding model

In [ ]:
embedding_model = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2',
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# this takes a few minutes on CPU but is much quicker on GPU
vector_database = FAISS.from_documents(passages_list, embedding_model)

## Cross encoder re-ranker model

In [ ]:
# ~30s
reranker_model = CrossEncoder("BAAI/bge-reranker-large")

In [ ]:
QUERY = "Who is Leopold Bloom's father?"
DOCS_TO_RETURN = 30

In [ ]:
retriever = vector_database.as_retriever(
    search_kwargs={'k': DOCS_TO_RETURN}
)
retrieved_passages = retriever.invoke(QUERY)

passages_contents = [p.page_content for p in retrieved_passages]
data = [(QUERY, content) for content in passages_contents]

In [ ]:
scores = reranker_model.predict(data)

In [ ]:
results_df = pd.DataFrame({
    'passage': passages_contents,
    'score': scores
})
results_df = results_df.sort_values(by='score', ascending=False)
results_df

,passage,score
4,He thought that he thought that he was a jew w...,0.904907
0,How did these beliefs and practices now appear...,0.642891
8,drunken goy ever. So you catch no money.\n\nBL...,0.493208
1,But sir Leopold was passing grave maugre his w...,0.134021
6,"fanciers, millwrights, newspaper canvassers,\n...",0.053500
3,"BLOOM: O, I so want to be a mother. MRS THORNT...",0.007625
15,Now let us speak of that fellowship that was t...,0.006447
22,Did Bloom discover common factors of similarit...,0.006159
19,What cerebration accompanied his frequentative...,0.004982
28,What is the age of the soul of man? As she hat...,0.004861


In [ ]:
results_df['passage'].iloc[0]

'He thought that he thought that he was a jew whereas he knew that he knew that\n\nhe knew that he was not.\n\nWhat, the enclosures of reticence removed, were their respective parentages? Bloom, only born male transubstantial heir of Rudolf Virag (subsequently Rudolph Bloom) of Szombathely, Vienna, Budapest, Milan, London and Dublin and of Ellen Higgins, second daughter of Julius Higgins (born Karoly) and Fanny Higgins (born Hegarty). Stephen, eldest surviving male consubstantial heir of Simon Dedalus of Cork and Dublin and of Mary, daughter of Richard and Christina Goulding (born Grier).\n\nHad Bloom and Stephen been baptised, and where and by whom, cleric or\n\nlayman?\n\nBloom (three times), by the reverend Mr Gilmer Johnston M. A., alone, in the protestant church of Saint Nicholas Without, Coombe, by James O’Connor, Philip Gilligan and James Fitzpatrick, together, under a pump in the village of Swords, and by the reverend Charles Malone C. C., in the church of the Three Patrons, Ra

In [ ]:
results_df['passage'].iloc[1]

'How did these beliefs and practices now appear to him? Not more rational than they had then appeared, not less rational than other\n\nbeliefs and practices now appeared.\n\nWhat first reminiscence had he of Rudolph Bloom (deceased)? Rudolph Bloom (deceased) narrated to his son Leopold Bloom (aged 6) a retrospective arrangement of migrations and settlements in and between Dublin, London, Florence, Milan, Vienna, Budapest, Szombathely with statements of satisfaction (his grandfather having seen Maria Theresia, empress of Austria, queen of Hungary), with commercial advice (having taken care of pence, the pounds having taken care of themselves). Leopold Bloom (aged 6) had accompanied these narrations by constant consultation of a geographical map of Europe (political) and by suggestions for the establishment of affiliated business premises in the various centres mentioned.\n\nHad time equally but differently obliterated the memory of these migrations in\n\nnarrator and listener?\n\nIn nar

## LLM

In [ ]:
hf_token = userdata.get('HF_TOKEN')
os.environ['HF_TOKEN'] = hf_token

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-7B-Instruct-GPTQ-Int4"  # Replace with the exact model path or ID
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",  # Automatically assign to GPU if available
    torch_dtype="auto"  # Use appropriate precision for quantized models
)


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:5006: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [ ]:
best_passages = results_df['passage'].iloc[:5]

In [ ]:
def make_prompt(user_query: str, passages: list[str]) -> str:
    prompt = 'Based on these passages:\n\n'
    prompt += '\n\n'.join(passages)
    prompt += f'\n\nAnswer to question: {user_query}\n\n'
    return prompt

In [ ]:
llm_prompt = make_prompt(QUERY, best_passages)
inputs = tokenizer(llm_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Based on these passages:

He thought that he thought that he was a jew whereas he knew that he knew that

he knew that he was not.

What, the enclosures of reticence removed, were their respective parentages? Bloom, only born male transubstantial heir of Rudolf Virag (subsequently Rudolph Bloom) of Szombathely, Vienna, Budapest, Milan, London and Dublin and of Ellen Higgins, second daughter of Julius Higgins (born Karoly) and Fanny Higgins (born Hegarty). Stephen, eldest surviving male consubstantial heir of Simon Dedalus of Cork and Dublin and of Mary, daughter of Richard and Christina Goulding (born Grier).

Had Bloom and Stephen been baptised, and where and by whom, cleric or

layman?

Bloom (three times), by the reverend Mr Gilmer Johnston M. A., alone, in the protestant church of Saint Nicholas Without, Coombe, by James O’Connor, Philip Gilligan and James Fitzpatrick, together, under a pump in the village of Swords, and by the reverend Charles Malone C. C., in the church of the Th

In [ ]:
inputs = tokenizer("Who is Leopold Bloom's father in Ulysses?", return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
# outputs nonsense

Who is Leopold Bloom's father in Ulysses? In James Joyce's novel "Ulysses," Leopold Bloom's father is named David. David Bloom was a solicitor and a member of the United Hebrew Congregation of Dublin. He passed away before the events of the novel take place


In [ ]:
def answer_question(query: str, num_passages: int = 5, max_new_tokens=50) -> str:
    retrieved_passages = retriever.invoke(query)
    passages_contents = [p.page_content for p in retrieved_passages]
    data = [(QUERY, content) for content in passages_contents]
    scores = reranker_model.predict(data)
    results_df = pd.DataFrame({
      'passage': passages_contents,
      'score': scores
    })
    results_df = results_df.sort_values(by='score', ascending=False)
    best_passages = results_df['passage'].iloc[:num_passages]
    llm_prompt = make_prompt(query, best_passages)
    inputs = tokenizer(llm_prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    return tokenizer.decode(outputs[0], skip_special_tokens=True).removeprefix(llm_prompt)

In [ ]:
answer_question("How does Buck Mulligan look like?", max_new_tokens=200)

'According to the passage, Buck Mulligan is described as a "stately, plump" man who comes from the stairhead, carrying a bowl of lather on which a mirror and a razor lie crossed. He wears a yellow dressing gown that is not yet tied around his waist, and it is supported by the mild morning air. His face is described as being "equine in its length," meaning it is long and horse-like in shape. His hair is described as "light untonsured," meaning it is unpowdered and natural in color, "grained and hued like pale oak," suggesting it has a grainy texture and a light, oak-like color. \n\nOverall, Buck Mulligan appears to be a robust, well-built man with a somewhat formal and imposing appearance. He is also described as making rapid crosses in the air and gurgling, which suggests he is a religious figure, possibly a priest or someone who performs religious rituals. Additionally, he bless'

#### questions, answers with passages, answers without passages

In [ ]:
# ~15 min
questions = [
    "Who is Leopold Bloom's father in Ulysses?",
    "What does Buck Mulligan look like?",
    'What role does Stephen Dedalus play in Ulysses, and how is he connected to the protagonist, Leopold Bloom?'
]
RAG_output = [answer_question(question, max_new_tokens = 200) for question in questions]
LLM_inputs = [tokenizer(question, return_tensors = 'pt').to(device) for question in questions]
LLM_outputs = [model.generate(**inputs, max_new_tokens = 200) for inputs in LLM_inputs]
LLM_outputs = [tokenizer.decode(outputs[0], skip_special_tokens = True) for outputs in LLM_outputs]

In [ ]:
df = pd.DataFrame({
    'Questions': questions,
    'RAG_Output': RAG_output,
    'LLM_Output': LLM_outputs
})

df

,Questions,RAG_Output,LLM_Output
0,Who is Leopold Bloom's father in Ulysses?,"Based on the passage, Leopold Bloom's father i...",Who is Leopold Bloom's father in Ulysses? In J...
1,What does Buck Mulligan look like?,Buck Mulligan is described as a broad-shoulder...,What does Buck Mulligan look like? What is his...
2,What role does Stephen Dedalus play in Ulysses...,Stephen Dedalus does not appear in the provide...,What role does Stephen Dedalus play in Ulysses...


In [ ]:
RAG_output[0]

'Based on the passage, Leopold Bloom\'s father is Rudolph Bloom (also referred to as Rudolf Virag). This can be inferred from the following excerpt:\n\n"Bloom, only born male transubstantial heir of Rudolf Virag (subsequently Rudolph Bloom) of Szombathely, Vienna, Budapest, Milan, London and Dublin and of Ellen Higgins, second daughter of Julius Higgins (born Karoly) and Fanny Higgins (born Hegarty)."\n\nThis passage explicitly states that Rudolf Virag is the father of Rudolph Bloom, who is in turn the father of Leopold Bloom. Therefore, Rudolph Bloom is Leopold Bloom\'s father in *Ulysses*. \n\nAdditionally, the dialogue where Rudolph Bloom addresses Leopold Bloom as "my son Leopold" further confirms this relationship. \n\nSo, to directly answer the question: **Leopold Bloom\'s father in *Ulysses* is Rudolph Bloom (R'

In [ ]:
LLM_outputs[0]

'Who is Leopold Bloom\'s father in Ulysses? In James Joyce\'s novel "Ulysses," Leopold Bloom\'s father is named David. David Bloom was a Jewish grocer who died before the events of the novel take place. Bloom refers to his father as "the old man" and describes him with some fondness, but also with a sense of loss, as he had passed away by the time the story begins.\n\nDavid Bloom is not a central character in the novel, as he does not appear in the narrative. However, Bloom often reflects on his father and his legacy, particularly in relation to his own identity and experiences as a Jewish man in early 20th century Dublin. The relationship between father and son is an important theme in the novel, exploring themes of loss, memory, and the transmission of identity across generations.'

In [ ]:
RAG_output[1]

"Buck Mulligan is described as a broad-shouldered, deep-chested, strong-limbed, frank-eyed, red-haired, freely-freckled, long-headed, deep-voiced, bare-kneed, shaggy-bearded, wide-mouthed, brawny-handed, hair-legged, ruddy-faced, sinewy-armed hero. He measures several ells from shoulder to shoulder and his rock-like, mountainous knees are covered with a strong growth of tawny prickly hair similar to the mountain gorse (Ulex Europeus). His nostrils are wide and cavernous, capable of lodging a fieldlark's nest. His eyes are large, with a teardrop and a smile constantly competing for dominance. They are the size of a good-sized cauliflower. A powerful current of warm breath issues from his mouth at regular intervals, and the loud, strong, hale resonances of his heart thunder rumblingly, causing the ground, the"

In [ ]:
LLM_outputs[1]

'What does Buck Mulligan look like? What is his attitude towards the world around him?\n\n1 Answer | Add Yours\n\nmwestwood\'s profile pic\n\nPosted on\n\nJames Joyce\'s Ulysses is a stream of consciousness novel in which the thoughts of the characters are recorded as they occur. Thus, it is not often that the author describes the physical appearance of the characters.\n\nIn Chapter 3, Buck Mulligan is described by Stephen Dedalus as he is walking down the street. He is wearing a white shirt and a green coat. When Stephen approaches Buck, the latter looks at him with an "amused, pitying glance." Buck is a character who is aware of his own beauty and his body; he is a man who is self-absorbed, confident, and carefree. His attitude toward the world around him is one of amusement and even disdain. He is also a man who is very proud of his body, as he is described as having a "slender, well-shaped figure." In fact,'

--------------------------- testing other books ------------------------------------------

In [ ]:
documents_loaders: list[UnstructuredFileLoader] = []

for filename in os.listdir(INPUT_PATH):
    if not filename.endswith('shakespeare_romeo_and_juliet.pdf'):
        continue

    documents_loaders.append(UnstructuredFileLoader(
        os.path.join(INPUT_PATH, filename),
        post_processors=post_processors,
        paragraph_grouper=group_broken_paragraphs,
    ))

<ipython-input-8-2bc7efee49b2>:7: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-unstructured package and should be used instead. To use it run `pip install -U :class:`~langchain-unstructured` and import as `from :class:`~langchain_unstructured import UnstructuredLoader``.
  documents_loaders.append(UnstructuredFileLoader(


In [ ]:
chunk_size = 800

text_splitter = RecursiveCharacterTextSplitter(
    separators=['\n\n'],
    chunk_size=chunk_size,
    chunk_overlap=200,
    length_function=len,
    keep_separator=False,
    is_separator_regex=False,
)